In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data_directory = "sequence_resolution_plots/"

In [3]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

create_directory(data_directory)

cuda:0


In [4]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 25
SEQ_END = 1800

In [5]:
TR_COLS = BASIC_TRAINING_COLS

In [6]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=TR_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=500, cols=TR_COLS)

100%|██████████| 10/10 [00:00<00:00, 121.41it/s]


# MASS TRAINING

In [7]:
INPUT_DIM = len(TR_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5
network_params = (INPUT_DIM, HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

In [8]:
labels = [1, 2, 3, 4]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)

for step, label in zip([1, 2, 3, 4], labels):    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_resolution", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END, 
                                                                   step_size=step)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_resolution"] = str(label)
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"mass_stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 968.91it/s]
Train_loss (1.11)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.39it/s] 
Train_loss (1.16)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.52it/s] 
Train_loss (1.15)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.58it/s] 
Train_loss (1.14)	 Train_acc (60.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  3.10it/s] 
Train_loss (1.14)	 Train_acc (70.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:00<00:00,  3.05it/s] 
Train_loss (1.14)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  3.07it/s] 
Train_loss (1.11)	 Train_acc (70.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  4.65it/s] 
Train_loss (1.16)	 Train_acc (60.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:00<00:00,  4.63it/s] 
Train_loss (1.16)	 Train_acc (40.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  4.61it/s] 
Train_loss (1.11)	 Train_acc (80.

# FORCE TRAINING

In [9]:
labels = [1, 2, 3, 4]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA, device=device)

for step, label in zip([1, 2, 3, 4], labels):    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_resolution", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_end=SEQ_END, 
                                                                   step_size=step)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_resolution"] = str(label)
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"force_stats.h5", key="stats")

100%|██████████| 10/10 [00:00<00:00, 1116.28it/s]
Train_loss (1.09)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.55it/s] 
Train_loss (1.13)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:01<00:00,  1.53it/s] 
Train_loss (1.09)	 Train_acc (70.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:01<00:00,  1.51it/s] 
Train_loss (1.10)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  3.06it/s] 
Train_loss (1.14)	 Train_acc (60.00)	 Val_acc (10.00): 100%|██████████| 2/2 [00:00<00:00,  3.03it/s] 
Train_loss (1.10)	 Train_acc (70.00)	 Val_acc (40.00): 100%|██████████| 2/2 [00:00<00:00,  2.99it/s] 
Train_loss (1.10)	 Train_acc (60.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  4.34it/s] 
Train_loss (1.16)	 Train_acc (40.00)	 Val_acc (20.00): 100%|██████████| 2/2 [00:00<00:00,  4.65it/s] 
Train_loss (1.12)	 Train_acc (40.00)	 Val_acc (30.00): 100%|██████████| 2/2 [00:00<00:00,  4.64it/s] 
Train_loss (1.08)	 Train_acc (60